In [1]:
import sys
import torch

sys.path.append('../')

/home/tsa87/anaconda3/envs/tacogfn/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from src.tacogfn.envs import frag_mol_env
from src.tacogfn.models import pharmaco_cond_graph_transformer
from src.tacogfn.data.utils import hetero_batch_to_batch
from src.tacogfn.data.utils import merge_pocket_and_molecule_data_list
import utils

Pharmacophore graphs

In [3]:
ctx = frag_mol_env.FragMolBuildingEnvContext()
molecule_data_list = utils.get_example_molecule_datalist(ctx)[:10]

In [4]:
pharmacophore_dataset = utils.get_example_pharmacophore_datalist()
pharmaco_batch = merge_pocket_and_molecule_data_list(pharmacophore_dataset, molecule_data_list)

In [5]:
model = pharmaco_cond_graph_transformer.PharmacophoreConditionalGraphTransformer(
    pharmacophore_dim=64,
    x_dim=ctx.num_node_dim,
    e_dim=ctx.num_edge_dim,
    g_dim=ctx.num_cond_dim,
    num_emb=64,
    num_layers=3,
    num_heads=2,
    ln_type="pre",
)

In [6]:
mol_g = hetero_batch_to_batch(pharmaco_batch, "compound")
pharmaco_g = hetero_batch_to_batch(pharmaco_batch, "pocket")

In [7]:
_, _ = model(mol_g, pharmaco_g)

In [12]:
import torch_geometric.data as gd
len(gd.Batch.from_data_list([pharmaco_batch[0]]))

1

Pocket graphs

In [8]:
import torch
from torch_geometric.data import HeteroData, Batch
from src.tacogfn.models.gvp_model import GVP_embedding
from torch_scatter import scatter_mean

In [9]:
model = pharmaco_cond_graph_transformer.PocketConditionalGraphTransformer(
    pocket_dim=128,
    x_dim=ctx.num_node_dim,
    e_dim=ctx.num_edge_dim,
    g_dim=ctx.num_cond_dim,
    num_emb=64,
    num_layers=3,
    num_heads=2,
    ln_type="pre",
)

In [10]:
pocket_dataset = utils.get_example_pockets()

In [11]:
pocket_batch = Batch.from_data_list(pocket_dataset)

In [12]:
molecule_batch = Batch.from_data_list(molecule_data_list)

In [13]:
model(
    molecule_batch,
    pocket_batch
)

(tensor([[-0.2520, -0.5245, -0.1991,  ...,  0.5002,  0.1795,  0.3489],
         [-0.2467, -0.3511, -0.2444,  ...,  0.5096,  0.2447,  0.3637],
         [-0.2256, -0.3796, -0.2176,  ...,  0.5257,  0.2423,  0.3858],
         ...,
         [-0.2646, -0.4371, -0.2848,  ...,  0.5225,  0.2175,  0.3837],
         [-0.2370, -0.4397, -0.2373,  ...,  0.5047,  0.2057,  0.3635],
         [-0.2460, -0.4401, -0.3196,  ...,  0.5172,  0.2013,  0.3289]],
        grad_fn=<SliceBackward0>),
 tensor([[-0.2368, -0.4315, -0.2440,  ...,  0.6721,  0.3047, -0.1275],
         [-0.2462, -0.4472, -0.2402,  ...,  0.6600,  0.3073, -0.0979],
         [-0.2382, -0.4497, -0.2311,  ...,  0.6670,  0.3193, -0.2114],
         ...,
         [-0.2277, -0.4486, -0.2393,  ...,  0.6343,  0.2984, -0.1577],
         [-0.2138, -0.4746, -0.2503,  ...,  0.6725,  0.2782, -0.1328],
         [-0.2443, -0.4762, -0.2457,  ...,  0.6690,  0.2969, -0.1298]],
        grad_fn=<CatBackward0>))